In [5]:
import pandas as pd
from sqlalchemy import create_engine

# Path to your SQLite database
db_path = "stock_data.db"
engine = create_engine(f"sqlite:///{db_path}")


In [37]:
# Load all data for MSFT
df = pd.read_sql("SELECT * FROM stock_data WHERE ticker='IBIT' ORDER BY date", engine)
print(df.tail())


    ticker        date       open       high        low      close adj_close  \
375   IBIT  2025-07-14  69.169998  69.459999  67.735001  68.180000      None   
376   IBIT  2025-07-15  66.930000  67.400002  65.769997  66.160004      None   
377   IBIT  2025-07-16  67.550003  68.239998  67.141998  67.830002      None   
378   IBIT  2025-07-17  67.035004  68.214996  66.820000  67.650002      None   
379   IBIT  2025-07-18  67.535004  67.837502  66.544998  66.680000      None   

         volume  macd macd_signal  ... bbands_middle bbands_lower stoch_k  \
375  68391600.0  None        None  ...          None         None    None   
376  82079400.0  None        None  ...          None         None    None   
377  58927000.0  None        None  ...          None         None    None   
378  45310300.0  None        None  ...          None         None    None   
379  49758396.0  None        None  ...          None         None    None   

    stoch_d   cci   adx   obv ema_20 ema_50   roc  
375 

In [33]:
import pandas as pd
from sqlalchemy import create_engine

# Load tickers.csv
df = pd.read_csv("etf.csv")
# Normalize column names (optional)
df.columns = df.columns.str.strip().str.lower()

# Connect to SQLite and write table
engine = create_engine("sqlite:///stock_data.db")
df.to_sql("symbol", engine, if_exists="replace", index=False)
print(f"Inserted {len(df)} rows into 'tickers' table.")


Inserted 2164 rows into 'tickers' table.


In [35]:
tickers = pd.read_sql("SELECT * FROM symbol", engine)
print(tickers.head())

   unnamed: 0 symbol
0           0   SPDV
1           1   EEMD
2           2    ZIG
3           3   HDGE
4           4   DWSH


In [7]:
# Load MSFT data for 2020 only
query = """
    SELECT * FROM stock_data
    WHERE ticker = 'TLT'
    AND date >= '2020-01-01' AND date <= '2020-12-31'
    ORDER BY date
"""
df_range = pd.readl_sql(query, engine)
print(df_range)


AttributeError: module 'pandas' has no attribute 'readl_sql'

In [12]:
import pandas as pd
import requests
from io import StringIO

url = "http://www.nasdaqtrader.com/dynamic/SymDir/etf.txt"
response = requests.get(url)
lines = response.text.strip().splitlines()

# Filter out non-data lines
data_lines = [line for line in lines if '|' in line and not line.startswith('#')]
clean_text = '\n'.join(data_lines)

df = pd.read_csv(StringIO(clean_text), sep='|')

df.columns = df.columns.str.strip()   # <<--- Strip whitespace from columns

# Remove empty columns and rows
df = df.dropna(how="all", axis=1)
if 'Symbol' in df.columns:
    df = df[df['Symbol'].astype(str).str.strip().str.len() > 0]

    # Show first 10 tickers and names
    print(df[['Symbol', 'Security Name']].head(10))

    # Get all ETF tickers
    etf_tickers = df['Symbol'].tolist()
else:
    print("Column 'Symbol' not found! Check your DataFrame columns:", df.columns.tolist())


Column 'Symbol' not found! Check your DataFrame columns: []


In [16]:
!pip install investpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 12.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for investpy: filename=investpy-1.0.8-py3-none-any.whl size=4481641 sha256=7d9a85cad7d451723daedc65eb54c774c7f0e46016cfdcdcfbe457f0d2b25275
  Stored in directory: /home/zhubo/.cache/pip/wheels/db/f4/ae/980b93b0257620bc8e4afc98a854a7a746a33eb8335fd07906
Successfully built investpy


In [18]:
len(etfs)

2164

In [29]:
import investpy
etfs = investpy.etfs.get_etfs(country='united states')
etfs.to_csv('all_etf.csv')

In [28]:
import pandas as pd
from sqlalchemy import create_engine

db_path = "stock_data.db"
engine = create_engine(f"sqlite:///{db_path}")

# Count number of unique tickers
df = pd.read_sql("SELECT COUNT(DISTINCT ticker) as num_stocks FROM stock_data", engine)
print(f"Number of unique stocks in DB: {df['num_stocks'][0]}")


Number of unique stocks in DB: 2148


In [31]:
"""
Batch Adds Technical Features to Each Stock in SQLite DB
--------------------------------------------------------
- Uses pandas-ta for classic technical indicators (MACD, RSI, ATR, etc.)
- Only adds columns if missing
- Only updates rows missing features (existing values not overwritten)
- No look-ahead bias or leakage
Requires: pandas, pandas-ta, sqlalchemy
"""

import pandas as pd
import pandas_ta as ta
from sqlalchemy import create_engine, text

# -- CONFIG --
db_path = "stock_data.db"
feature_columns = [
    'macd', 'macd_signal', 'macd_hist',
    'rsi', 'atr',
    'bbands_upper', 'bbands_middle', 'bbands_lower',
    'stoch_k', 'stoch_d',
    'cci', 'adx', 'obv', 'ema_20', 'ema_50', 'roc'
]

# -- 1. Connect to DB --
engine = create_engine(f"sqlite:///{db_path}")

# -- 2. Add missing feature columns to the table --
with engine.connect() as conn:
    existing_cols = pd.read_sql("PRAGMA table_info(stock_data);", conn)['name'].tolist()
with engine.begin() as conn:
    for feat in feature_columns:
        if feat not in existing_cols:
            print(f"Adding column {feat} ...")
            try:
                conn.execute(text(f"ALTER TABLE stock_data ADD COLUMN {feat} REAL"))
            except Exception as e:
                print(f"Failed to add {feat}: {e}")

# -- 3. Define the calculation function (no look-ahead bias) --
def calc_features(df):
    # MACD
    macd = ta.macd(df['close'])
    df['macd'] = macd.iloc[:, 0] if 'MACD_12_26_9' in macd.columns else macd.iloc[:, 0]
    df['macd_signal'] = macd.iloc[:, 1] if 'MACDs_12_26_9' in macd.columns else macd.iloc[:, 1]
    df['macd_hist'] = macd.iloc[:, 2] if 'MACDh_12_26_9' in macd.columns else macd.iloc[:, 2]
    # RSI
    df['rsi'] = ta.rsi(df['close'])
    # ATR
    df['atr'] = ta.atr(df['high'], df['low'], df['close'])
    # Bollinger Bands (robust to col names)
    bb = ta.bbands(df['close'])
    bb_upper = [col for col in bb.columns if col.startswith("BBU_")]
    bb_middle = [col for col in bb.columns if col.startswith("BBM_")]
    bb_lower = [col for col in bb.columns if col.startswith("BBL_")]
    df['bbands_upper'] = bb[bb_upper[0]] if bb_upper else None
    df['bbands_middle'] = bb[bb_middle[0]] if bb_middle else None
    df['bbands_lower'] = bb[bb_lower[0]] if bb_lower else None
    # Stochastic Oscillator
    stoch = ta.stoch(df['high'], df['low'], df['close'])
    stoch_k = [col for col in stoch.columns if col.startswith("STOCHk_")]
    stoch_d = [col for col in stoch.columns if col.startswith("STOCHd_")]
    df['stoch_k'] = stoch[stoch_k[0]] if stoch_k else None
    df['stoch_d'] = stoch[stoch_d[0]] if stoch_d else None
    # CCI
    df['cci'] = ta.cci(df['high'], df['low'], df['close'])
    # ADX
    adx = ta.adx(df['high'], df['low'], df['close'])
    adx_col = [col for col in adx.columns if col.startswith("ADX_")]
    df['adx'] = adx[adx_col[0]] if adx_col else None
    # OBV
    df['obv'] = ta.obv(df['close'], df['volume'])
    # EMA 20, EMA 50
    df['ema_20'] = ta.ema(df['close'], length=20)
    df['ema_50'] = ta.ema(df['close'], length=50)
    # ROC
    df['roc'] = ta.roc(df['close'])
    return df


# -- 4. Main loop: for each ticker, update only missing features --
tickers = pd.read_sql("SELECT DISTINCT ticker FROM stock_data", engine)['ticker'].tolist()
for ticker in tickers:
    print(f"\n{ticker}: Checking for missing features...")
    # Only load necessary columns
    df = pd.read_sql(
        f"SELECT date, open, high, low, close, volume, {','.join(feature_columns)} FROM stock_data WHERE ticker = ? ORDER BY date",
        engine, params=(ticker,))
    if df.empty:
        continue
    # Identify rows with missing features
    missing_mask = df[feature_columns].isnull().any(axis=1)
    if not missing_mask.any():
        print(f"  All features already present.")
        continue

    print(f"  Calculating features for {missing_mask.sum()} / {len(df)} rows...")
    # Calculate features (safe for look-ahead, uses only past and present)
    df_features = calc_features(df)

    # Only update rows that were missing
    to_update = df_features[missing_mask].copy()

    # Write back only needed features (do not overwrite existing values)
    with engine.begin() as conn:
        for _, row in to_update.iterrows():
            update_dict = {feat: None if pd.isna(row[feat]) else float(row[feat]) for feat in feature_columns}
            update_dict['ticker'] = ticker
            update_dict['date'] = row['date']
            set_clause = ', '.join([f"{feat} = :{feat}" for feat in feature_columns])
            conn.execute(
                text(f"""
                    UPDATE stock_data SET {set_clause}
                    WHERE ticker = :ticker AND date = :date
                """),
                update_dict
            )
    print(f"  Updated {len(to_update)} rows.")

print("\nAll done! All technical features are up-to-date in your database.")



AAAU: Checking for missing features...
  Calculating features for 1739 / 1739 rows...
  Updated 1739 rows.

AADR: Checking for missing features...
  Calculating features for 2513 / 2513 rows...
  Updated 2513 rows.

AAXJ: Checking for missing features...
  Calculating features for 2513 / 2513 rows...
  Updated 2513 rows.

ACES: Checking for missing features...
  Calculating features for 1771 / 1771 rows...
  Updated 1771 rows.

ACIO: Checking for missing features...
  Calculating features for 1514 / 1514 rows...
  Updated 1514 rows.

ACSG: Checking for missing features...
  Calculating features for 1662 / 1662 rows...
  Updated 1662 rows.

ACSI: Checking for missing features...
  Calculating features for 2184 / 2184 rows...
  Updated 2184 rows.

ACT: Checking for missing features...
  Calculating features for 962 / 962 rows...
  Updated 962 rows.

ACWF: Checking for missing features...
  Calculating features for 2513 / 2513 rows...
  Updated 2513 rows.

ACWI: Checking for missing feat

KeyboardInterrupt: 